In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from multiprocessing import Process, Manager, Pool
import numpy as np

### util method

In [2]:
def get_rating(id_trabalhador, escolaridade_trabalhador, graduacao_trabalhador, 
               cidade_trabalhador, escolaridade_vaga, graduacao_vaga, cidade_vaga, id_vaga):
    points = 0
    if escolaridade_trabalhador == escolaridade_vaga:
        points += 1
    if graduacao_trabalhador != np.nan and graduacao_trabalhador == graduacao_vaga:
        points += 1
    if cidade_trabalhador == cidade_vaga:
        points += 1
    
    return points
    #shared_list.append({'id_trabalhador': id_trabalhador, 'id_vaga':id_vaga, 'rating':rating})

## load trabalhadores

In [3]:
df_trabalhadores = pd.read_csv("data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_TRABALHADORES_SP.csv", sep=";", encoding="iso8859-1")

/home/chris/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_trabalhadores.dropna(subset=['PRETENSOES'], inplace=True)
df_trabalhadores[['PRETENSOES','LIXO']] = df_trabalhadores.PRETENSOES.str.split("(",1,expand=True)

In [5]:
#df_trabalhadores = df_trabalhadores[:30000]
df_trabalhadores = df_trabalhadores.sample(7000)
df_trabalhadores['id_trabalhador'] = df_trabalhadores.index
print(df_trabalhadores.shape)
df_trabalhadores.head()

(4000, 22)


,NACIONALIDADE,DEFICIENCIAS,BAIRRO,CEP,CODIGO_MUNICIPIO_IBGE,NOME_MUNICIPIO,UF,ESCOLARIDADE,ESTUDANTE,CURSOS_PROFISSIONALIZANTES,...,IDIOMAS,HABILITACAO,VEICULOS,DISP_VIAJAR,DISP_DORMIR_EMP,DISP_AUSENTAR_DOMIC,PRETENSOES,MUNICIPIOS_INTERESSE,LIXO,id_trabalhador
29636,IGNORADO,NaN,BELA VISTA,1.3112e+06,355030,SAO PAULO,SP,Superior Completo,N,NaN,...,NaN,Nenhum,N,N,N,N,142335-Analista de pesquisa de mercado,NaN,"N,94,0,Indiferente)",29636
3254047,BRASILEIRA,NaN,NaN,7.71467e+06,350900,CAIEIRAS,SP,Médio Completo,N,NaN,...,NaN,Nenhum,N,N,N,N,784205-Alimentador de linha de produção,NaN,"N,28,0,Indiferente)|914405-Auxiliar de mecânic...",3254047
3603160,BRASILEIRA,NaN,ALTO INDUSTRIAL,9.7612e+06,354870,SAO BERNARDO DO CAMPO,SP,Médio Completo,N,NaN,...,NaN,Nenhum,N,N,N,N,422310-OPERADOR DE TELEMARKETING ATIVO E RECEP...,NaN,"N,84,0,Indiferente)",3603160
1950664,BRASILEIRA,NaN,MONTANHAO,9.78345e+06,354870,SAO BERNARDO DO CAMPO,SP,Médio Completo,N,NaN,...,NaN,Nenhum,N,S,S,S,253115-Publicitário,NaN,"N,13,0,Indiferente)|262410-Desenhista industri...",1950664
1047152,BRASILEIRA,NaN,JARDIM IMPERATRIZ,6859400,352220,ITAPECERICA DA SERRA,SP,Médio Incompleto,S,NaN,...,NaN,Nenhum,N,N,N,N,411005-Auxiliar administrativo,NaN,"S,0,0,Indiferente)|422315-Atendente central te...",1047152


## load vagas

In [6]:
df_vagas = pd.read_csv("data/vagas_mock.csv")
#df_vagas = df_vagas[:10000]
df_vagas = df_vagas.sample(3000)
print(df_vagas.shape)
df_vagas.head()

(7000, 7)


,cidade,escolaridade,graduacao,id_empresa,industria,nome_empresa,posicao
88442,BOTUCATU,Mestrado,processos gerenciais,89b5308b-3a13-4b6f-91d7-d0b2ff67f674,Químico,engage e-business partnerships,514325-Trabalhador da manutenção de edificações
14946,MOGI GUACU,Médio Completo,Nenhum,50cea7d9-9230-4c4e-8758-98db4d0262e4,Elétrico,generate virtual interfaces,521125-Repositor de mercadorias
62826,CAMPINAS,Nenhum,Nenhum,5f368d36-9a73-46ca-96c0-c1f684f5e6dd,Metal Mecânico,reinvent transparent mindshare,513505-Auxiliar de cozinha
67755,BAURU,Nenhum,Nenhum,02f6c0c6-d6cd-4f7d-978e-5801ee68858b,Logística,cultivate interactive systems,513505-Auxiliar nos serviços de alimentação
66930,ITAPEVI,Fundamental Completo,Nenhum,1a14be0a-d3d4-41f9-ad79-b75a75f603d6,Administrativo,re-intermediate B2B e-commerce,512105-Empregado doméstico nos serviços gerais


## Feature engineering
automaticamente "avalia" uma posicao para um trabalhador

In [7]:
rating_list = []
for index, row in tqdm(df_trabalhadores.iterrows(), total=df_trabalhadores.shape[0]):
    id_trabalhador = row['id_trabalhador']
    escolaridade_trabalhador = row['ESCOLARIDADE']
    graduacao_trabalhador = row['GRADUACOES']
    cidade_trabalhador = row['NOME_MUNICIPIO']
    
    for index_vaga, row_vaga in df_vagas.iterrows():
        escolaridade_vaga = row_vaga['escolaridade']
        graduacao_vaga = row_vaga['graduacao']
        cidade_vaga = row_vaga['cidade']
        id_vaga = row_vaga['id_empresa']
        
        
        rating = get_rating(id_trabalhador=id_trabalhador, escolaridade_trabalhador=escolaridade_trabalhador, graduacao_trabalhador=graduacao_trabalhador, 
               cidade_trabalhador=cidade_trabalhador, escolaridade_vaga=escolaridade_vaga, graduacao_vaga=graduacao_vaga, cidade_vaga=cidade_vaga, id_vaga=id_vaga)
        rating_list.append({'id_trabalhador':id_trabalhador, 'id_posicao':id_vaga, 'rating':rating})

In [8]:
del(df_trabalhadores)
del(df_vagas)

In [9]:
df_match = pd.DataFrame(rating_list)
print(df_match.shape)
df_match.head()

(28000000, 3)


,id_posicao,id_trabalhador,rating
0,89b5308b-3a13-4b6f-91d7-d0b2ff67f674,29636,0
1,50cea7d9-9230-4c4e-8758-98db4d0262e4,29636,0
2,5f368d36-9a73-46ca-96c0-c1f684f5e6dd,29636,0
3,02f6c0c6-d6cd-4f7d-978e-5801ee68858b,29636,0
4,1a14be0a-d3d4-41f9-ad79-b75a75f603d6,29636,0


In [10]:
del(rating_list)

In [11]:
df_match.rating.value_counts().plot.bar();

In [12]:
df_match.rating.value_counts()

0    18099338
1     9813880
2       86765
3          17
Name: rating, dtype: int64

In [13]:
df_match.to_csv("data/matches2.csv")